<a href="https://colab.research.google.com/github/geoUFSC/geostats/blob/main/BinWidthImpact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testing the impact of bin width

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Try to import ipywidgets for interactivity
widgets_ok = False
try:
    from ipywidgets import interact, IntSlider, Checkbox
    widgets_ok = True
except Exception:
    widgets_ok = False

# 1) Generate dataset
np.random.seed(123)
n_samples =150
mu, sigma = 35, 1
porosity = np.random.normal(mu, sigma, n_samples)
porosity = np.clip(porosity, 0, 65)

import os
os.makedirs("data", exist_ok=True)
csv_path = "data/soil_porosity_dataset.csv"

import pandas as pd
df = pd.DataFrame({"sample_id": np.arange(1, n_samples+1), "porosity": porosity})
df.to_csv(csv_path, index=False)

# 2) Bin recommendations
def freedman_diaconis_bins(data):
    q75, q25 = np.percentile(data, [75, 25])
    iqr = q75 - q25
    if iqr == 0:
        return 10
    h = 2 * iqr / (len(data) ** (1/3))
    if h <= 0:
        return 10
    return max(int(np.ceil((data.max() - data.min()) / h)), 5)

def scott_bins(data):
    s = np.std(data, ddof=1)
    if s == 0:
        return 10
    h = 3.5 * s / (len(data) ** (1/3))
    if h <= 0:
        return 10
    return max(int(np.ceil((data.max() - data.min()) / h)), 5)

def sturges_bins(n):
    return max(int(np.ceil(np.log2(n) + 1)), 5)

fd_bins = freedman_diaconis_bins(df["porosity"].values)
sc_bins = scott_bins(df["porosity"].values)
st_bins = sturges_bins(len(df))

# 3) Plot function
def plot_hist(bins=20, density=True, show_mean=True, show_std=True):
    plt.figure(figsize=(8,5))
    plt.hist(df["porosity"], bins=bins, density=density, edgecolor="black")
    plt.xlabel("Soil Porosity (%)")
    plt.ylabel("Probability" if density else "Frequency")
    plt.title("Histogram of Soil Porosity")
    if show_mean:
        plt.axvline(df["porosity"].mean(), linestyle="--", linewidth=2, label="Mean")
    if show_std:
        m = df["porosity"].mean(); s = df["porosity"].std(ddof=1)
        plt.axvline(m - s, linestyle=":", linewidth=1.5, label="±1 SD")
        plt.axvline(m + s, linestyle=":", linewidth=1.5)
    txt = f"Recommended bins: \nFD={fd_bins}, Scott={sc_bins}, Sturges={st_bins}"
    plt.text(0.02, 0.98, txt, transform=plt.gca().transAxes, va="top")
    if density:
        plt.ylim(bottom=0, top=1)
    else:
        plt.ylim(bottom=0, top=n_samples+1)
    plt.grid(True)
    plt.legend()
    plt.show()

if widgets_ok:
    interact(plot_hist,
             bins=IntSlider(value=20, min=1, max=60, step=1, description="# of Bins"),
             density=Checkbox(value=True, description="Normalize (Probability)"),
             show_mean=Checkbox(value=True, description="Show Mean"),
             show_std=Checkbox(value=True, description="Show ±1 SD"))
else:
    plot_hist(bins=20, density=True, show_mean=True, show_std=True)

